# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating arguments with your requests. 

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.  

In this chat scenario, as the user talks back and forth with the bot, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. 

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.6.3"
#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.6.3

Let's define a prompt outlining a dialogue chat bot.

In [2]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var executionSettings = new OpenAIPromptExecutionSettings
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

Register your semantic function

In [3]:
var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

Initialize your arguments

In [4]:
var history = "";
var arguments = new KernelArguments()
{
    ["history"] = history
};

Chat with the Bot

In [5]:
var userInput = "Hi, I'm looking for a sci-fi book suggestions";
arguments["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(kernel, arguments);

Update the history with the output and set this as the new input value for the next request

In [6]:
history += $"\nUser: {userInput}\nAI: {bot_answer}\n";
arguments["history"] = history;

Console.WriteLine(history);


User: Hi, I'm looking for a sci-fi book suggestions
AI: Hello! I'd be happy to help you with that. What kind of science fiction books are you interested in? Are you looking for something more along the lines of space opera, dystopian futures, time travel, or something else? Let me know so I can give you some tailored recommendations.



Keep Chatting!

In [7]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the arguments
    arguments["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(kernel, arguments);

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {answer}\n";
    history += result;

    arguments["history"] = history;

    // Show the response
    Console.WriteLine(result);
};

In [8]:
await Chat("I would like books about dystopian futures set in space.");


User: I would like books about dystopian futures set in space.
AI: Great choice! Here are some science fiction books about dystopian futures set in space that you might enjoy:

1. "The Expanse" series by James S.A. Corey - This series follows a future where humanity has colonized the solar system, leading to political tensions and conflicts that threaten to erupt into all-out war.

2. "Red Rising" by Pierce Brown - Set in a future where society is divided into color-coded castes, this series follows a young man's journey to overthrow the oppressive ruling class.

3. "The Stars My Destination" by Alfred Bester - This classic novel follows the story of a man seeking revenge in a future where teleportation technology has transformed society.

I hope you find these recommendations helpful! Let me know if you'd like more suggestions or if you have any specific preferences.



In [9]:
await Chat("that sounds interesting, can you recommend other similar books?");


User: that sounds interesting, can you recommend other similar books?
AI: Sure! Here are some more science fiction books about dystopian futures set in space that you might enjoy:

1. "Altered Carbon" by Richard K. Morgan - In a future where consciousness can be transferred between bodies, a former soldier is hired to solve a murder mystery that leads him into a world of corruption and intrigue.

2. "The Hunger Games" by Suzanne Collins - Set in a dystopian future where children are forced to fight to the death in a televised competition, this series follows a young girl's struggle for survival and rebellion against the oppressive government.

3. "The Long Way to a Small, Angry Planet" by Becky Chambers - This novel follows the crew of a spaceship as they travel through space, encountering different alien species and facing challenges that test their bonds and beliefs.

I hope you find these additional recommendations enjoyable! Let me know if you'd like more suggestions or if you hav

In [10]:
await Chat("Which book from the ones you listed do you think most people find interesting?");


User: Which book from the ones you listed do you think most people find interesting?
AI: It's difficult to say which book most people find interesting, as individual preferences can vary greatly. However, "The Hunger Games" by Suzanne Collins is a widely popular series that has captured the attention of many readers with its gripping storyline and themes of survival and rebellion. It has also been adapted into successful movies, further increasing its popularity.



In [11]:
await Chat("could you list some more books I could read about the topic(s) you mentioned?");


User: could you list some more books I could read about the topic(s) you mentioned?
AI: I'm glad you're interested in exploring more books on the topic! Here are some additional science fiction books about dystopian futures set in space that you might find intriguing:

1. "Ender's Game" by Orson Scott Card - This novel follows a young boy named Ender Wiggin who is recruited into a military training program to prepare for an alien invasion.

2. "Dune" by Frank Herbert - Set in a distant future where noble houses vie for control of a desert planet called Arrakis, this epic novel explores themes of politics, religion, and ecology.

3. "Hyperion" by Dan Simmons - In this novel, a group of pilgrims journey to the distant planet of Hyperion, each with their own story to tell and secrets to uncover.

4. "The Left Hand of Darkness" by Ursula K. Le Guin - This novel follows a human envoy's mission to a planet where the inhabitants can change gender at will, exploring themes of gender and polit

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [12]:
Console.WriteLine(history);


User: Hi, I'm looking for a sci-fi book suggestions
AI: Hello! I'd be happy to help you with that. What kind of science fiction books are you interested in? Are you looking for something more along the lines of space opera, dystopian futures, time travel, or something else? Let me know so I can give you some tailored recommendations.

User: I would like books about dystopian futures set in space.
AI: Great choice! Here are some science fiction books about dystopian futures set in space that you might enjoy:

1. "The Expanse" series by James S.A. Corey - This series follows a future where humanity has colonized the solar system, leading to political tensions and conflicts that threaten to erupt into all-out war.

2. "Red Rising" by Pierce Brown - Set in a future where society is divided into color-coded castes, this series follows a young man's journey to overthrow the oppressive ruling class.

3. "The Stars My Destination" by Alfred Bester - This classic novel follows the story of a m